# Sentiment Analysis using local model service

We're utilizing the `/sentiment` endpoint in out model api service to analyze the reviews and assign a sentiment value. <br/>

Using the [nlptown/bert-base-multilingual-uncased-sentiment](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment) model downloaded from Huggingface

### Defining utility function to perform REST calls

In [1]:
import requests
import json

def post_json_request(url, payload, additional_headers=None):
    # Set default headers
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
    }

    # Merge with additional headers if provided
    if additional_headers:
        headers.update(additional_headers)
        
    payload_json = json.dumps(payload)

    # Perform the POST request
    response = requests.post(url, data=payload_json, headers=headers)

    # Return the response
    return response



## Spark

### Defining the spark session

In [2]:
from pyspark.sql import SparkSession,  DataFrame, functions as F
spark = SparkSession.builder.appName("sentiment_analysis").getOrCreate()

24/01/24 19:42:40 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### Defining the Dataframe by reading Iceberg table stored in S3/Minio

In [3]:
df = spark.read.table("lakehouse.kaggle_airbnb.reviews")

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


### Filtering for just the first few records, just for testing

In [4]:
df_5 = df.limit(5)
df_5.toPandas().head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...


### Defining Spark UDF to call Model REST API

In [5]:
from pyspark.sql.functions import col, udf

def get_sentiment(review:str)->str:
    URL = "http://model-api-svc.models.svc.cluster.local:8000/api/v1/models/sentiment"
    request = {"text":review}
    response = post_json_request(URL,  request).json()
    return response['result']
    
get_sentiment_udf = udf(lambda r: get_sentiment(r)) 


### Setting new column (review_sentiment) to the UDF, and caching to avoid repeated UDF calls 

In [6]:
df_5_sentiments = df_5.withColumn('review_sentiment',  get_sentiment_udf(col("comments"))  ).cache() 
df_5_sentiments.select('comments', 'review_sentiment').toPandas().head()

,comments,review_sentiment
0,Cute and cozy place. Perfect location to every...,Very Positive
1,Kelly has a great room in a very central locat...,Very Positive
2,"Very spacious apartment, and in a great neighb...",Positive
3,Close to Seattle Center and all it has to offe...,Very Positive
4,Kelly was a great host and very accommodating ...,Very Positive


In [8]:
df_5_sentiments.select('comments', 'review_sentiment').show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------+
|comments                                                                                                                                                                                                                                                                                                                                                                                                      |review_sentiment|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
spark.stop()